## Analisi Canale di Twitch

Twitch è una piattaforma di streaming i quali contenuti sono principalmente videoludici.
Anche Francesco Ingold, studente del gruppo, è un creatore di contenuti per la piattaforma e con la pandemia avuta all'inizio del 2020 il suo canale è diventato un più serio.
Per questo motivo abbiamo deciso di fare un'analisi dei dati dell'analitica del suo canale, forniti direttamente da twitch.
Al momento il dataset contiene circa ##inserirenumeroassolutamentesiinserireilnumero di righe.
ogni riga rappresenta una giornata a partire dal #giornopreciso maggio 2018, per cui le giornate in cui non è avvenuto uno stream all'interno del dataset hanno i dati inizializzati a zero, ma sono presenti.

In [4]:
import pandas as pd;
import plotly.express as px;
import numpy as np;

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


#AGGIORNARE CATEGORIE DA (COMPRESO) 5 NOVEMBRE

df = pd.read_csv("Analitica_canale.csv",index_col='Data', parse_dates=True)
df

,PubblicitÃ (minuti),Durata annunci (secondi) per ora,Spettatori medi,Messaggi chat,Utenti chat,Visualizzazioni clip,Clip create,Segue,Spettatori Raid e Host (%),Visualizzazioni live,...,Entrate pubblicitÃ,Ricavi dalla vendita di giochi,Entrate dalle estensioni,Ricavi promozioni,Abbonamenti Prime,Abbonamenti paganti in totale,Abbonati Liv. 1,Abbonamenti regalo in totale,Abbonamenti Liv. 1 in regalo,Categoria
Data,,,,,,,,,,,,,,,,,,,,,
2018-05-26,0,0.0,0.00,0,0,0,0,0,0.00,0,...,0.000,0,0,0,0,0,0,0,0,NaN
2018-05-27,0,0.0,1.50,0,0,0,0,1,0.00,17,...,0.000,0,0,0,0,0,0,0,0,NaN
2018-05-28,0,0.0,0.00,0,0,0,0,0,0.00,0,...,0.000,0,0,0,0,0,0,0,0,NaN
2018-05-29,0,0.0,0.08,0,0,0,0,0,0.00,1,...,0.000,0,0,0,0,0,0,0,0,NaN
2018-05-30,0,0.0,0.00,0,0,0,0,0,0.00,0,...,0.000,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-31,0,0.0,4.03,268,11,0,0,0,1.12,65,...,0.028,0,0,0,0,0,0,0,0,Splatoon
2020-11-01,0,0.0,0.00,0,0,0,1,0,0.00,0,...,0.000,0,0,0,0,0,0,0,0,NaN
2020-11-02,0,0.0,4.81,251,11,0,4,0,10.75,69,...,0.014,0,0,0,1,0,0,0,0,TSW2


## Dataset
all'interno del dataset abbiamo accesso alle colonne presenti qui sotto. la colonna chiamata "Categoria" non era presente all'interno dell'analitica di twitch, ma l'abbiamo dovuta aggiungere noi.
avremmo voluto fare lo stesso lavoro con i messaggi della chat durante la live, ma purtroppo quei dati erano impossibili da avere.

In [5]:
#Tutte le colonne disponibili
df.columns

Index(['PubblicitÃ  (minuti)', 'Durata annunci (secondi) per ora',
       'Spettatori medi', 'Messaggi chat', 'Utenti chat',
       'Visualizzazioni clip', 'Clip create', 'Segue',
       'Spettatori Raid e Host (%)', 'Visualizzazioni live', 'Spettatori max',
       'Minuti visti', 'Minuti di streaming', 'Spettatori unici',
       'Entrate degli abbonamenti', 'Ricavi Prime',
       'Entrate degli abbonamenti in regalo', 'Ricavi bit',
       'Entrate pubblicitÃ ', 'Ricavi dalla vendita di giochi',
       'Entrate dalle estensioni', 'Ricavi promozioni', 'Abbonamenti Prime',
       'Abbonamenti paganti in totale', 'Abbonati Liv. 1',
       'Abbonamenti regalo in totale', 'Abbonamenti Liv. 1 in regalo',
       'Categoria'],
      dtype='object')

## Categorie e giorni
visto che abbiamo aggiunto le categorie, la prima cosa che abbiamo controllato è che percentuale di esse effettivamente non fosse assente dal dataset. come potete notare ##inserirequipercentualeaggiornata che è circa la metà.
Abbiamo deciso di aggregare i dati in trimestri, per avere una miglior visualizzazione di come si è evoluto nel tempo il canale del mio compagno, in particolar modo per quanto riguarda i minuti di streaming fatti e quelli visti dagli spettatori.
come potete notare nel tempo il canale ha avuto una crescita sia in quantità di streaming fatte, che in minuti visti dagli spettatori.
si può notare un primo aumento in concomitanza con il primo lockdown in italia, che ha portato la gente ad avere un po' di tempo libero da passare in casa.
un dato "outline" è rappresentato da una grande live fatta da Francesco durante il suo compleanno, dove ha streammato per 12 ore.

In [7]:
#groupby giorni
df["giornoSet"] = df.index.dayofweek
df["giornoSet"] = df["giornoSet"].map({0:"Lun",1:"Mar",2:"Mer",3:"Gio",4:"Ven",5:"Sab",6:"Dom"})


def raggruppaMesi(time):
    grouped="Q"
    quarter=(int(time.strftime("%m"))-1)//3+1
    grouped=grouped+str(quarter)+"-"
    grouped=grouped+time.strftime("%Y")
    return grouped
    
df["Periodo"]=df.index.map(raggruppaMesi)

#dataframe contenente esclusivamente i giorni in cui ho streammato
dfStr = df[df['Minuti di streaming']>0]



#Check quante hanno la categoria
categs = dfStr["Categoria"].notnull()
px.histogram(categs).show()

#tempo di streaming over time
px.line(df["Minuti di streaming"]).show()


px.line(dfStr["Minuti visti"]).show()


## Streaming durante la settimana
Ci siamo chiesti se un dato giorno della settimana influenzasse il quantitativo di streaming portato dal mio compagno, e quanto di esso venisse visto dagli spettatori. come si può vedere dal grafico non ci sono trend particolari legati a questi dati.

In [8]:

px.box(dfStr,x="giornoSet",y="Minuti di streaming",category_orders={"giornoSet":["Lun","Mar","Mer","Gio","Ven","Sab","Dom"]},title="Minuti di streaming nei giorni della settimana")

In [9]:
px.box(dfStr,x="giornoSet",y="Spettatori medi",category_orders={"giornoSet":["Lun","Mar","Mer","Gio","Ven","Sab","Dom"]},title="Spettatori medi nei giorni della settimana")

## Elaborazione categorie
Prima di fare analisi più approfondite sulle categorie, abbiamo dovuto elaborarle.
visto che in uno stesso stream ci possono essere più categorie, i dati venivano particolarmente frammentati, per cui abbiamo dovuto dividere le categorie.
L'obbiettivo è stato quello di creare un dataframe in cui ogni riga contenesse solo una categoria, e i dati della giornata di stream leggermente modificati.
per fare questo lavoro, inizialmente abbiamo eliminato tutte le righe che non avevano alcuna categoria, per poter così fare una pulizia iniziale dei dati.
dopodichè abbiamo creato un altro dataframe, in cui inserire i dati modificati.
abbiamo studiato quali delle colonne del dataframe di partenza ci potevano servire. mentre alcune colonne andavano bene già così e non sono state toccate(ad esempio il dato sugli "spettatori medi"). altre colonne, invece, hanno dovuto subire un'approssimazione sui dati, semplicemente dividendole per il numero di categorie portate quel giorno.
le colonne toccate in questa maniera sono "minuti di streaming" e "minuti visti"
per esempio, semplificando, se avessimo nel dataframe di partenza la seguente situazione
## nonso come fare
Data           Minuti di streaming     Spettatori medi    Categoria
09-03-2019     120                     12                 Isaac - PPT

nel nuovo dataset diviene
Data           Minuti di streaming     Spettatori medi    Categoria
09-03-2019     60                      12                 Isaac
09-03-2019     60                      12                 PPT

In [10]:
#inizio lavoro da macellaio 
dfMac =dfStr[dfStr["Categoria"].notnull()]

mask =dfMac["Categoria"].str.count("-")>0


listOfRows = dfMac[mask]

dfMac = dfMac[ np.invert(mask)]


for i in range(len(listOfRows)):
    row = listOfRows.iloc[[i]]

    categories = row["Categoria"].str.split("-",expand=True)
    
    for ind in range(len(categories.columns)):
        newRow = row.copy()
        newRow['Categoria'] =  categories[ind].str.strip()
        newRow["Minuti visti"]  = newRow["Minuti visti"]/len(categories)
        newRow["Minuti di streaming"] = newRow["Minuti di streaming"]/len(categories)
        dfMac = dfMac.append(newRow,ignore_index=False) 
        
dfMac
        

,PubblicitÃ (minuti),Durata annunci (secondi) per ora,Spettatori medi,Messaggi chat,Utenti chat,Visualizzazioni clip,Clip create,Segue,Spettatori Raid e Host (%),Visualizzazioni live,...,Entrate dalle estensioni,Ricavi promozioni,Abbonamenti Prime,Abbonamenti paganti in totale,Abbonati Liv. 1,Abbonamenti regalo in totale,Abbonamenti Liv. 1 in regalo,Categoria,giornoSet,Periodo
Data,,,,,,,,,,,,,,,,,,,,,
2019-02-17,0,0.0,4.02,12,3,0,0,1,37.29,77,...,0,0,0,0,0,0,0,PPT,Dom,Q1-2019
2019-02-18,0,0.0,6.35,75,5,6,1,3,52.37,101,...,0,0,0,0,0,0,0,T99,Lun,Q1-2019
2019-02-20,0,0.0,2.43,24,5,1,0,2,9.92,35,...,0,0,0,0,0,0,0,T99,Mer,Q1-2019
2019-02-23,0,0.0,4.51,4,3,2,1,1,0.00,40,...,0,0,0,0,0,0,0,T99,Sab,Q1-2019
2019-02-24,0,0.0,1.83,39,4,19,1,2,0.21,41,...,0,0,0,0,0,0,0,T99,Dom,Q1-2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-18,0,0.0,5.21,304,11,0,0,0,4.98,97,...,0,0,0,0,0,0,0,PPT,Dom,Q4-2020
2020-10-18,0,0.0,5.21,304,11,0,0,0,4.98,97,...,0,0,0,0,0,0,0,T99,Dom,Q4-2020
2020-10-18,0,0.0,5.21,304,11,0,0,0,4.98,97,...,0,0,0,0,0,0,0,Teffect,Dom,Q4-2020


## Categorie più streammate
dopo aver elaborato le categorie, abbiamo potuto lavorare in maniera comoda con esse. la prima domanda che ci siamo posti è stata quale categoria fossero quelle più portate, e che quindi fossero più importanti per dare un'impronta di distinzione al canale.

In [11]:
dfCat = dfMac.groupby(["Categoria"],as_index=False).sum();
dfCat.sort_values(by=["Minuti di streaming"], ascending=True)
fig = px.bar(dfCat.nlargest(n=10,columns= "Minuti di streaming",keep="first").sort_values(by=["Minuti di streaming"], ascending=True), x= "Minuti di streaming", y = "Categoria",title="Categorie più streammate")
fig.show()

## Categorie più viste
Dopodichè ci siamo concentrati sulle categorie più viste dagli spettatori. abbiamo scelto di visualizzarle in uno scatterplot per poter individuare in maniera più facile quali categorie sono viste di più. se una categoria ha i punti distribuiti mediamente più in alto di un altra si nota subito, anche se una categoria magari è stata portata meno spesso.

In [12]:
fig = px.scatter(dfMac, y= "Spettatori medi", x = "Categoria",title="Categorie più viste")
fig.show()

## Interact
Infine questo notebook ha anche lo scopo di essere uno strumento per il mio compagno per poter controllare l'andamento del suo canale anche in futuro. per questa ragione abbiamo lavorato a un sistema che permettesse di eseguire diverse operazioni come regressioni lineari potendo scegliere dinamicamente le variabili da utilizzare. questo permette di individuare diversi trend e in caso in cui il canale perda visibilità sarebbe subito facile individuarne le cause.
##magaridirequalcosariguardantealcunigraficifighidiquestasezione

In [9]:

def sceltaTipo(tipo_grafico):
    if tipo_grafico=="scatter":
        px.scatter(dfStr, y= "Spettatori medi", x = "Periodo",title="Progresso spettatori medi per periodo(scatter)").show()
    elif tipo_grafico=="box":
        px.box(dfStr, y= "Spettatori medi", x = "Periodo",title="Progresso spettatori medi per periodo(box)").show()

interact(sceltaTipo,tipo_grafico=["scatter","box"])

print()


interactive(children=(Dropdown(description='tipo_grafico', options=('scatter', 'box'), value='scatter'), Outpu…

In [10]:
#px.histogram(dfCat,x=dfCat.index,y=['Entrate pubblicitÃ\xa0', 'Entrate degli abbonamenti',"Ricavi Prime",'Ricavi bit']).show()


In [15]:
#Work in Progress



import sklearn
import sklearn.linear_model

import plotly.offline as py
import plotly.graph_objs as go

from datetime import datetime

def f(x,inverti_assi):
    x_axis= [dfMac.index.astype(np.int64)/((10**9)*3600*24)-17943,dfMac["Minuti di streaming"].values,(dfMac["Minuti visti"]/dfMac["Spettatori unici"]).values]

    reg = sklearn.linear_model.LinearRegression()
    x=x_axis[x];
    y=dfMac["Spettatori medi"].values
    if inverti_assi:
        temp=x
        x=y
        y=temp
    
    m = len(x) # Numero di osservazioni
    xm = np.reshape(x,(m,1))
    
    reg.fit(xm,y)
    
    w=reg.coef_
    b=reg.intercept_
    
    x_w=np.array([np.min(xm),np.max(xm)])
    y_w=w*x_w+b
    
    data=[
        go.Scatter(x=x,y=y,mode="markers",name="data"),
        go.Scatter(x=x_w,y=y_w,mode="lines",name="regression")
    ]
    
    layout = go.Layout(
        xaxis = dict(title="variabile esplicativa"),
        yaxis = dict(title="Spect medi : variabile target")
    )

    fig=go.Figure(data,layout)
    py.iplot(fig)


interact(f, x=[('giorni',0),('minuti str',1),("minuti visti per persona",2)],inverti_assi=False);


interactive(children=(Dropdown(description='x', options=(('giorni', 0), ('minuti str', 1), ('minuti visti per …